In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import csv
import sys
import os
import h5py
import simplejson as json

import sys
sys.path.append('../src')
import data_utils_opp as data_utils

In [2]:
#load the data
data_path = '/data2/data/zebrahim/OpportunityUCIDataset'

dr = data_utils.data_reader('opportunity', data_path)

78
reading file 1 of 19
reading file 2 of 19
reading file 3 of 19
reading file 4 of 19
reading file 5 of 19
reading file 6 of 19
reading file 7 of 19
reading file 8 of 19
reading file 9 of 19
reading file 10 of 19
reading file 11 of 19
reading file 12 of 19
reading file 13 of 19
reading file 14 of 19
reading file 15 of 19
reading file 16 of 19
reading file 17 of 19
reading file 18 of 19
reading file 19 of 19
reading file 1 of 1
reading file 1 of 4
reading file 2 of 4
reading file 3 of 4
reading file 4 of 4


In [3]:
dr.data['training']['inputs'].shape, dr.data['training']['targets'].shape

((700165, 77), (700165,))

In [4]:
train_data = np.expand_dims(dr.data['training']['inputs'] , axis=0)
validation_data = np.expand_dims(dr.data['validation']['inputs'] , axis=0)
test_data = np.expand_dims(dr.data['test']['inputs'] , axis=0)

In [5]:
train_gt = np.zeros(dr.data['training']['targets'].shape, dtype='float32')
idx_change_activity = np.where(dr.data['training']['targets'][:-1] - dr.data['training']['targets'][1:] != 0)[0]
for idx in idx_change_activity:
    interval = range(max(idx - 16,0), min(idx + 16, train_gt.shape[0]))
    train_gt[interval] = 1


In [6]:
validation_gt = np.zeros(dr.data['validation']['targets'].shape, dtype='float32')
idx_change_activity = np.where(dr.data['validation']['targets'][:-1] - dr.data['validation']['targets'][1:] != 0)[0]
for idx in idx_change_activity:
    interval = range(max(idx - 16,0), min(idx + 16, validation_gt.shape[0]))
    validation_gt[interval] = 1

In [7]:
test_gt = np.zeros(dr.data['test']['targets'].shape, dtype='float32')
idx_change_activity = np.where(dr.data['test']['targets'][:-1] - dr.data['test']['targets'][1:] != 0)[0]
for idx in idx_change_activity:
    interval = range(max(idx - 16,0), min(idx + 16, test_gt.shape[0]))
    test_gt[interval] = 1

In [8]:
train_gt = np.expand_dims(train_gt , axis=0)
validation_gt = np.expand_dims(validation_gt , axis=0)
test_gt = np.expand_dims(test_gt , axis=0)

train_gt = np.expand_dims(train_gt , axis=-1)
validation_gt = np.expand_dims(validation_gt , axis=-1)
test_gt = np.expand_dims(test_gt , axis=-1)

In [9]:
np.unique(dr.data['test']['targets'])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18])

In [10]:
data = {}
data['train_data'] = train_data
data['train_gt'] = train_gt

data['validation_data'] = validation_data
data['validation_gt'] = validation_gt


data['test_data'] =  test_data
data['test_gt'] = test_gt

In [12]:
# change point detection groun truth, there is 1 in transition states and (-16,16) time steps. Others are 0.
import hickle
path_to_save = '../data/'
with open(path_to_save+'opp3.hkl', 'w') as fout:
    hickle.dump(data, fout)